Endemic Animal WebScraper

In [27]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import time 
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException, ElementNotInteractableException, StaleElementReferenceException, NoSuchElementException

# Path to your chromedriver executable
import pandas as pd
from bs4 import BeautifulSoup
import os
import requests

In [29]:
# Path to your chromedriver executable
service = Service(executable_path="chromedriver.exe")
driver = webdriver.Chrome(service=service)

data = pd.read_csv('animal_data.csv', encoding='ISO-8859-1')

# Slice the DataFrame to start from the 10th row (index 9)
data = data.iloc[28:]

# Creating the main 'images' directory
main_folder_name = 'images'
if not os.path.isdir(main_folder_name):
    os.makedirs(main_folder_name)
    
def download_image(url, full_folder_path, num):
        # Write image to file
        response = requests.get(url)
        if response.status_code == 200:
            with open(os.path.join(full_folder_path, f"{num}.jpg"), 'wb') as file:
                file.write(response.content)

for index, row in data.iterrows():
    col = row['Animal Name']
    print(f"Animal Name {col}")
    animal_name = col
    animal_name_search = animal_name.replace(" ", "+")
    print(animal_name_search)
    # Define a subfolder inside the 'images' folder
    subfolder_name = animal_name  # Replace 'subfolder' with your desired subfolder name
    full_folder_path = os.path.join(main_folder_name, subfolder_name)
    # Create the subfolder if it doesn't exist
    if not os.path.isdir(full_folder_path):
        os.makedirs(full_folder_path)

    animal_link = f"https://www.google.com/search?q={animal_name_search}&source=lnms&tbm=isch"
    driver.get(animal_link)
    time.sleep(10)

    # Continuously scroll down until the bottom of the page is reached
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(10)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # a = input("Waiting for user to start...")
    time.sleep(20)
    driver.execute_script("window.scrollTo(0,0);")

    elements = driver.find_elements(By.XPATH, "//div[@class='eA0Zlc WghbWd FnEtTd mkpRId m3LIae RLdvSe qyKxnc ivg-i PZPZlf GMCzAd']")
    len_elements = len(elements)

    print(f"Found {len_elements} image containers")

    for i in range(len_elements):
        if i >= 400:
            print("Finished")
            break

        try:
            element = elements[i]
            
            # Scroll element into view
            driver.execute_script("arguments[0].scrollIntoView(true);", element)
            
            # Locate the inner element that contains the text
            # elem = driver.find_element(By.XPATH, f"(//div[@class='eA0Zlc WghbWd FnEtTd mkpRId m3LIae RLdvSe qyKxnc ivg-i PZPZlf GMCzAd']//div[@class='toI8Rb OSrXXb'])[{i+1}]")
            try:
                elem = WebDriverWait(driver, 30).until(
                    EC.presence_of_element_located((By.XPATH, f"(//div[@class='eA0Zlc WghbWd FnEtTd mkpRId m3LIae RLdvSe qyKxnc ivg-i PZPZlf GMCzAd']//div[@class='toI8Rb OSrXXb'])[{i+1}]"))
                )
            except (TimeoutException, NoSuchElementException) as e:
                print(f"Element not found: {str(e)}")
                continue
            # Check if the text in 'elem' contains the value of 'animal_name'
            if animal_name.lower() in elem.text.lower():  # Using lower() to make the comparison case-insensitive
                print(f"Found matching element with text: {elem.text}")
                
                # Wait until the element is clickable
                # elem_img = WebDriverWait(driver, 40).until(
                #     EC.element_to_be_clickable((By.XPATH, f"(//div[@class='eA0Zlc WghbWd FnEtTd mkpRId m3LIae RLdvSe qyKxnc ivg-i PZPZlf GMCzAd']//img[@class='YQ4gaf'])[{i+1}]"))
                # )
                # elem_imgURL = elem_img.get_attribute("src")
                # # Click on the element
                # element.click()
                
                # Increase timeout duration and add handling for stale elements
                try:
                    elem_img = WebDriverWait(driver, 60).until(
                        EC.element_to_be_clickable((By.XPATH, f"(//div[@class='eA0Zlc WghbWd FnEtTd mkpRId m3LIae RLdvSe qyKxnc ivg-i PZPZlf GMCzAd']//img[@class='YQ4gaf'])[{i+1}]"))
                    )
                    elem_imgURL = elem_img.get_attribute("src")
                    element.click()
                    # driver.execute_script("arguments[0].click();", element)
                except (TimeoutException, StaleElementReferenceException) as e:
                    print(f"Error while trying to click on element {i+1}: {str(e)}")
                    continue

                
                timeStarted = time.time()
                time.sleep(10)
                while True:
                    img_elem = driver.find_element(By.XPATH, "//div[@class='p7sI2 PUxBg']//img[@class='sFlh5c FyHeAf']")
                    img_URL = img_elem.get_attribute("src")
                    
                    if img_URL != elem_imgURL:
                        print(f"Clicked on element {i+1} with matching text: {img_URL}")
                        break
                    else:
                        currentTime = time.time()
                        if currentTime - timeStarted > 10:
                            print("Timeout! Will download a lower resolution image and move onto the next one")
                            break
                        
                try:
                    download_image(img_URL, full_folder_path, i)
                    print("Downloaded element %s out of %s total. URL: %s" % (i, len_elements + 1, img_URL))
                except:
                    print("Couldn't download an image %s, continuing downloading the next one"%(i))
            else:
                print(f"No match found in element {i+1}: {elem.text}")
            
        except (ElementNotInteractableException, ElementClickInterceptedException) as e:
            print(f"Could not click element {i+1}: {str(e)}")
            continue
        
        time.sleep(2)  # Wait a bit before moving to the next element


time.sleep(10)
driver.quit()

Animal Name Palawan Binturong
Palawan+Binturong
Found 559 image containers
Found matching element with text: Palawan binturong - Wikipedia


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='p7sI2 PUxBg']//img[@class='sFlh5c FyHeAf']"}
  (Session info: chrome=128.0.6613.86); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF68B149632+30946]
	(No symbol) [0x00007FF68B0FE3C9]
	(No symbol) [0x00007FF68AFF6FDA]
	(No symbol) [0x00007FF68B04822C]
	(No symbol) [0x00007FF68B04850C]
	(No symbol) [0x00007FF68B08DCB7]
	(No symbol) [0x00007FF68B06CAAF]
	(No symbol) [0x00007FF68B08B041]
	(No symbol) [0x00007FF68B06C813]
	(No symbol) [0x00007FF68B03A6E5]
	(No symbol) [0x00007FF68B03B021]
	GetHandleVerifier [0x00007FF68B27F83D+1301229]
	GetHandleVerifier [0x00007FF68B28BDB7+1351783]
	GetHandleVerifier [0x00007FF68B282A03+1313971]
	GetHandleVerifier [0x00007FF68B17DD06+245686]
	(No symbol) [0x00007FF68B10758F]
	(No symbol) [0x00007FF68B1